In [1]:
import os
import argparse
import pandas as pd
import numpy as np
import xgboost as xgb
from math import ceil
from math import sqrt
from math import acos
from operator import itemgetter
from itertools import product
from scipy.stats import pearsonr
import matplotlib.pyplot as plt
import gc
import seaborn as sns
from scipy import stats
from collections import defaultdict
from tqdm import tqdm
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe , space_eval
import time

# Training new models on chymotrypsin-immunopeptides

In [2]:
def load_data(vector_filename, ion_type):
    # Read file
    if vector_filename.split(".")[-1] == "pkl":
        vectors = pd.read_pickle(vector_filename)
    elif vector_filename.split(".")[-1] == "h5":
        # vectors = pd.read_hdf(vector_filename, key='table', stop=1000)
        vectors = pd.read_hdf(vector_filename, key="table")
    else:
        print("Unsuported feature vector format")
        exit(1)

    # Extract targets for given ion type
    target_names = list(vectors.columns[vectors.columns.str.contains("targets")])
    if not "targets_{}".format(ion_type) in target_names:
        print("Targets for {} could not be found in vector file.".format(ion_type))
        print("Vector file only contains these targets: {}".format(target_names))
        exit(1)

    targets = vectors.pop("targets_{}".format(ion_type))
    target_names.remove("targets_{}".format(ion_type))
    for n in target_names:
        vectors.pop(n)

    # Get psmids
    psmids = vectors.pop("psmid")

    return (vectors, targets, psmids)

In [3]:
def get_params_combinations(params):
    keys, values = zip(*params.items())
    combinations = [dict(zip(keys, v)) for v in product(*values)]
    return(combinations)

In [4]:
def get_best_params(df, params_grid):
    params = {}
    best = df[df['test-rmse-mean'] == df['test-rmse-mean'].min()]
    best_rmse = df['test-rmse-mean'].min()
    for p in params_grid.keys():
        params[p] = best[p].iloc[0]
    # num_boost_round = best['boosting-round'].iloc[0]
    return(params, best_rmse)

In [5]:
def gridsearch(xtrain, params, params_grid):
    cols = ['boosting-round', 'test-rmse-mean', 'test-rmse-std', 'train-rmse-mean', 'train-rmse-std']
    cols.extend(sorted(params_grid.keys()))
    result = pd.DataFrame(columns=cols)

    count = 1
    combinations = get_params_combinations(params_grid)

    for param_overrides in combinations:
        print("Working on combination {}/{}".format(count, len(combinations)))
        count += 1
        params.update(param_overrides)
        tmp = xgb.cv(params, xtrain, nfold=5, num_boost_round=200, early_stopping_rounds=10, verbose_eval=10)
        tmp['boosting-round'] = tmp.index
        for param in param_overrides.keys():
            tmp[param] = param_overrides[param]
        result = result.append(tmp)

    print("Grid search ready!\n")

    return(result)

In [6]:
def ms2pip_pearson(true, pred):
    """
    Return pearson of tic-normalized, log-transformed intensities, 
    the MS2PIP way.
    """
    #tic_norm = lambda x: x / np.sum(x)
    # log_transform = lambda x: np.log2(x + 0.001)
    corr = pearsonr(
        true, 
        pred
    )[0]
    return (corr)

In [7]:
def spectral_angle(true, pred, epsilon=1e-7):
    """
    Return square root normalized spectral angle.
    See https://doi.org/10.1074/mcp.O113.036475
    """
    
    de_log = lambda x: (2**x)-0.001
    l2_normalize = lambda x: x / sqrt(max(sum(x**2), epsilon))
    
    pred_norm = l2_normalize(de_log(pred))
    true_norm = l2_normalize(de_log(true))
    
    spectral_angle = 1 - (2 * acos(np.dot(pred_norm, true_norm)) / np.pi)

    return (spectral_angle)

# New scoring method

In [8]:
class Scorer:
    def __init__(self,psmids):
        self.psmids = psmids
        
    def psm_score(self,targets, predictions):
        tmp = pd.DataFrame(columns=["psmids", "targets", "predictions"])
        tmp["psmids"] = np.array(self.psmids)
        tmp["targets"] = np.array(targets)
        tmp["predictions"] = np.array(predictions)
        tmp2 = tmp.groupby("psmids").agg({'predictions': list, 'targets': list}).reset_index()
        spectral_corr = []
        pearson_corr = []
        for spectra in range(0, len(tmp2["psmids"])):
            spectral_corr.append(spectral_angle(np.array(tmp2.targets.loc[spectra]), np.array(tmp2.predictions.loc[spectra])))
            pearson_corr.append(ms2pip_pearson(np.array(tmp2.targets.loc[spectra]), np.array(tmp2.predictions.loc[spectra])))
        return (pearson_corr, spectral_corr) 

# Hyperparameter optimization B ions

In [9]:
vectors, targets, psmids = load_data("data/immunopeptide_chymotrypsin.pkl", "B")

In [33]:
len(psmids.unique())

520579

In [13]:
upeps = psmids.unique()
np.random.shuffle(upeps)
test_psms = upeps[:int(len(upeps) * 0.1)]

train_vectors = vectors[~psmids.isin(test_psms)]
train_targets = targets[~psmids.isin(test_psms)]
train_psmids = psmids[~psmids.isin(test_psms)]

test_vectors = vectors[psmids.isin(test_psms)]
test_targets = targets[psmids.isin(test_psms)]
test_psmids = psmids[psmids.isin(test_psms)]

In [14]:
pd.Series(test_psmids.unique()).str.contains('PXD010154').sum()

5960

In [15]:
pd.Series(train_psmids.unique()).str.contains('PXD010154').sum()

54428

In [16]:
xtrain = xgb.DMatrix(train_vectors, label=train_targets)
xtest = xgb.DMatrix(test_vectors, label=test_targets)
evallist = [(xtrain, 'train'),(xtest, 'test')]

In [19]:
space= {
    'eta': hp.loguniform('eta', np.log(0.01), np.log(1)),
    'max_depth': hp.quniform('max_depth', 3, 18, 1),
    'max_leaves': hp.quniform('max_leaves', 5, 500, 1),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1.0),
    'reg_lambda': hp.uniform('reg_lambda', 0.0, 1.0),
    'gamma' : hp.uniform ('gamma', 0.0,1),
    'min_child_weight' : hp.quniform('min_child_weight', 0, 500, 1),
    'subsample': hp.quniform('subsample', 0.5, 1 ,0.1),
    'reg_alpha': hp.quniform('reg_alpha', 0, 5 ,0.1)
}

In [22]:
def objective(space):
    params = {
    "nthread": 64,
    "objective": "reg:squarederror",
    "eval_metric": 'rmse',
    "eta": space["eta"],
    "max_depth": int(space['max_depth']),
    "grow_policy":"lossguide",
    "max_leaves":int(space["max_leaves"]),
    "min_child_weight": int(space["min_child_weight"]),
    "gamma": space['gamma'],
    "reg_lambda" : space['reg_lambda'],
    "colsample_bytree": space['colsample_bytree'],
    "subsample" : space["subsample"],
    "reg_alpha" : space["reg_alpha"]
    }
    print(params)
    tmp = xgb.cv(params, xtrain, nfold=4, num_boost_round=400, early_stopping_rounds=10, verbose_eval=True)
    rmse = tmp['test-rmse-mean'].min()

    print ("rmse:", rmse)
    return {'loss': rmse, 'status': STATUS_OK, "params": params}

In [23]:
trials = Trials()
best_hyperparams = fmin(fn = objective,
                        space = space,
                        algo = tpe.suggest,
                        max_evals = 25,
                        trials = trials)


best_hyperparams

{'nthread': 64, 'objective': 'reg:squarederror', 'eval_metric': 'rmse', 'eta': 0.19930579190931164, 'max_depth': 11, 'grow_policy': 'lossguide', 'max_leaves': 475, 'min_child_weight': 305, 'gamma': 0.572100811979309, 'reg_lambda': 0.9647796716443167, 'colsample_bytree': 0.6310208275906879, 'subsample': 0.6000000000000001, 'reg_alpha': 0.2}
[0]	train-rmse:7.59217+0.00037	test-rmse:7.59221+0.00090

[1]	train-rmse:6.11557+0.00161	test-rmse:6.11578+0.00197

[2]	train-rmse:4.93899+0.00195	test-rmse:4.93933+0.00205

[3]	train-rmse:4.00398+0.00179	test-rmse:4.00418+0.00176

[4]	train-rmse:3.26443+0.00143	test-rmse:3.26486+0.00158

[5]	train-rmse:2.68346+0.00128	test-rmse:2.68409+0.00145

[6]	train-rmse:2.23058+0.00159	test-rmse:2.23146+0.00177

[7]	train-rmse:1.87950+0.00179	test-rmse:1.88067+0.00208

[8]	train-rmse:1.61204+0.00198	test-rmse:1.61361+0.00242

[9]	train-rmse:1.41136+0.00184	test-rmse:1.41333+0.00251

[10]	train-rmse:1.26542+0.00206	test-rmse:1.26778+0.00274

[11]	train-rmse:1.1

[268]	train-rmse:0.75649+0.00119	test-rmse:0.79532+0.00019

[269]	train-rmse:0.75630+0.00118	test-rmse:0.79525+0.00021

[270]	train-rmse:0.75613+0.00123	test-rmse:0.79518+0.00022

[271]	train-rmse:0.75594+0.00122	test-rmse:0.79510+0.00023

[272]	train-rmse:0.75580+0.00120	test-rmse:0.79504+0.00022

[273]	train-rmse:0.75564+0.00125	test-rmse:0.79498+0.00022

[274]	train-rmse:0.75552+0.00123	test-rmse:0.79493+0.00021

[275]	train-rmse:0.75528+0.00123	test-rmse:0.79481+0.00025

[276]	train-rmse:0.75512+0.00120	test-rmse:0.79474+0.00024

[277]	train-rmse:0.75497+0.00122	test-rmse:0.79468+0.00025

[278]	train-rmse:0.75473+0.00116	test-rmse:0.79458+0.00021

[279]	train-rmse:0.75455+0.00117	test-rmse:0.79450+0.00019

[280]	train-rmse:0.75433+0.00116	test-rmse:0.79440+0.00016

[281]	train-rmse:0.75420+0.00117	test-rmse:0.79436+0.00016

[282]	train-rmse:0.75399+0.00121	test-rmse:0.79427+0.00021

[283]	train-rmse:0.75383+0.00118	test-rmse:0.79420+0.00022

[284]	train-rmse:0.75362+0.00117	test-rm

[0]	train-rmse:9.27642+0.00032	test-rmse:9.27685+0.00106                       

[1]	train-rmse:9.11491+0.00034	test-rmse:9.11532+0.00105                       

[2]	train-rmse:8.95691+0.00029	test-rmse:8.95699+0.00105                       

[3]	train-rmse:8.80080+0.00035	test-rmse:8.80114+0.00096                       

[4]	train-rmse:8.64833+0.00030	test-rmse:8.64838+0.00100                       

[5]	train-rmse:8.49788+0.00029	test-rmse:8.49817+0.00097                       

[6]	train-rmse:8.35089+0.00031	test-rmse:8.35073+0.00095                       

[7]	train-rmse:8.20597+0.00029	test-rmse:8.20596+0.00094                       

[8]	train-rmse:8.06341+0.00031	test-rmse:8.06359+0.00098                       

[9]	train-rmse:7.92361+0.00027	test-rmse:7.92381+0.00095                       

[10]	train-rmse:7.78636+0.00029	test-rmse:7.78658+0.00095                      

[11]	train-rmse:7.65151+0.00024	test-rmse:7.65171+0.00090                      

[12]	train-rmse:7.51914+0.00

[202]	train-rmse:0.86156+0.00044	test-rmse:0.87556+0.00060                     

[203]	train-rmse:0.85978+0.00041	test-rmse:0.87387+0.00062                     

[204]	train-rmse:0.85801+0.00042	test-rmse:0.87219+0.00061                     

[205]	train-rmse:0.85633+0.00041	test-rmse:0.87060+0.00062                     

[206]	train-rmse:0.85467+0.00040	test-rmse:0.86903+0.00064                     

[207]	train-rmse:0.85302+0.00038	test-rmse:0.86748+0.00067                     

[208]	train-rmse:0.85140+0.00041	test-rmse:0.86596+0.00062                     

[209]	train-rmse:0.84982+0.00042	test-rmse:0.86448+0.00062                     

[210]	train-rmse:0.84830+0.00045	test-rmse:0.86305+0.00059                     

[211]	train-rmse:0.84686+0.00045	test-rmse:0.86169+0.00060                     

[212]	train-rmse:0.84543+0.00043	test-rmse:0.86037+0.00063                     

[213]	train-rmse:0.84400+0.00039	test-rmse:0.85903+0.00067                     

[214]	train-rmse:0.84264+0.0

[0]	train-rmse:3.22583+0.00026	test-rmse:3.22605+0.00051                       

[1]	train-rmse:1.52134+0.00843	test-rmse:1.52171+0.00898                       

[2]	train-rmse:1.18222+0.00483	test-rmse:1.18275+0.00505                       

[3]	train-rmse:1.11637+0.00648	test-rmse:1.11695+0.00676                       

[4]	train-rmse:1.09414+0.00473	test-rmse:1.09479+0.00486                       

[5]	train-rmse:1.07578+0.00530	test-rmse:1.07647+0.00505                       

[6]	train-rmse:1.05787+0.00468	test-rmse:1.05862+0.00495                       

[7]	train-rmse:1.04562+0.00359	test-rmse:1.04649+0.00365                       

[8]	train-rmse:1.03590+0.00389	test-rmse:1.03689+0.00448                       

[9]	train-rmse:1.02607+0.00171	test-rmse:1.02714+0.00296                       

[10]	train-rmse:1.01909+0.00195	test-rmse:1.02020+0.00298                      

[11]	train-rmse:1.01189+0.00359	test-rmse:1.01306+0.00323                      

[12]	train-rmse:1.00619+0.00

...

[113]	train-rmse:0.70967+0.00046	test-rmse:0.78899+0.00056                    

[114]	train-rmse:0.70899+0.00039	test-rmse:0.78884+0.00057                    

[115]	train-rmse:0.70833+0.00039	test-rmse:0.78870+0.00058                    

[116]	train-rmse:0.70785+0.00040	test-rmse:0.78858+0.00054                    

[117]	train-rmse:0.70725+0.00036	test-rmse:0.78849+0.00051                    

[118]	train-rmse:0.70670+0.00023	test-rmse:0.78839+0.00051                    

[119]	train-rmse:0.70618+0.00020	test-rmse:0.78830+0.00052                    

[120]	train-rmse:0.70558+0.00041	test-rmse:0.78814+0.00054                    

[121]	train-rmse:0.70499+0.00065	test-rmse:0.78803+0.00056                    

[122]	train-rmse:0.70461+0.00076	test-rmse:0.78796+0.00057                    

[123]	train-rmse:0.70401+0.00077	test-rmse:0.78784+0.00054                    

[124]	train-rmse:0.70353+0.00084	test-rmse:0.78777+0.00054                    

[125]	train-rmse:0.70291+0.00071	test-rm

[317]	train-rmse:0.61742+0.00059	test-rmse:0.77916+0.00061                    

[318]	train-rmse:0.61705+0.00057	test-rmse:0.77915+0.00060                    

[319]	train-rmse:0.61655+0.00050	test-rmse:0.77914+0.00060                    

[320]	train-rmse:0.61621+0.00058	test-rmse:0.77914+0.00059                    

[321]	train-rmse:0.61589+0.00065	test-rmse:0.77914+0.00059                    

[322]	train-rmse:0.61552+0.00062	test-rmse:0.77912+0.00057                    

[323]	train-rmse:0.61511+0.00065	test-rmse:0.77910+0.00056                    

[324]	train-rmse:0.61473+0.00065	test-rmse:0.77909+0.00056                    

[325]	train-rmse:0.61432+0.00073	test-rmse:0.77908+0.00056                    

[326]	train-rmse:0.61399+0.00077	test-rmse:0.77907+0.00055                    

[327]	train-rmse:0.61361+0.00076	test-rmse:0.77907+0.00056                    

[328]	train-rmse:0.61332+0.00081	test-rmse:0.77907+0.00055                    

[329]	train-rmse:0.61306+0.00087	test-rm

[139]	train-rmse:0.77910+0.00034	test-rmse:0.80228+0.00063                     

[140]	train-rmse:0.77875+0.00038	test-rmse:0.80205+0.00060                     

[141]	train-rmse:0.77836+0.00038	test-rmse:0.80177+0.00059                     

[142]	train-rmse:0.77800+0.00035	test-rmse:0.80152+0.00058                     

[143]	train-rmse:0.77762+0.00037	test-rmse:0.80126+0.00059                     

[144]	train-rmse:0.77722+0.00029	test-rmse:0.80099+0.00062                     

[145]	train-rmse:0.77683+0.00036	test-rmse:0.80072+0.00060                     

[146]	train-rmse:0.77649+0.00038	test-rmse:0.80049+0.00061                     

[147]	train-rmse:0.77618+0.00036	test-rmse:0.80030+0.00062                     

[148]	train-rmse:0.77577+0.00035	test-rmse:0.80001+0.00061                     

[149]	train-rmse:0.77541+0.00037	test-rmse:0.79977+0.00063                     

[150]	train-rmse:0.77500+0.00031	test-rmse:0.79950+0.00066                     

[151]	train-rmse:0.77463+0.0

[341]	train-rmse:0.73371+0.00021	test-rmse:0.77529+0.00087                     

[342]	train-rmse:0.73352+0.00023	test-rmse:0.77518+0.00083                     

[343]	train-rmse:0.73337+0.00020	test-rmse:0.77510+0.00085                     

[344]	train-rmse:0.73327+0.00021	test-rmse:0.77506+0.00084                     

[345]	train-rmse:0.73315+0.00021	test-rmse:0.77500+0.00083                     

[346]	train-rmse:0.73301+0.00022	test-rmse:0.77494+0.00084                     

[347]	train-rmse:0.73286+0.00023	test-rmse:0.77487+0.00081                     

[348]	train-rmse:0.73277+0.00023	test-rmse:0.77483+0.00083                     

[349]	train-rmse:0.73259+0.00023	test-rmse:0.77474+0.00084                     

[350]	train-rmse:0.73242+0.00027	test-rmse:0.77466+0.00087                     

[351]	train-rmse:0.73231+0.00025	test-rmse:0.77461+0.00088                     

[352]	train-rmse:0.73217+0.00023	test-rmse:0.77455+0.00086                     

[353]	train-rmse:0.73203+0.0

{'colsample_bytree': 0.891381182690278,
 'eta': 0.08060612330262913,
 'gamma': 0.031142279181653326,
 'max_depth': 18.0,
 'max_leaves': 117.0,
 'min_child_weight': 500.0,
 'reg_alpha': 3.4000000000000004,
 'reg_lambda': 0.27245538266226343,
 'subsample': 0.7000000000000001}

In [24]:
hyperoptimization_results = {'test-rmse-mean': [x['loss'] for x in trials.results]}
for key in trials.results[0]["params"].keys():
    hyperoptimization_results[key] = [x['params'][key] for x in trials.results]

In [25]:
df = pd.DataFrame(hyperoptimization_results)

In [26]:
df.head()

,test-rmse-mean,nthread,objective,eval_metric,eta,max_depth,grow_policy,max_leaves,min_child_weight,gamma,reg_lambda,colsample_bytree,subsample,reg_alpha
0,0.787612,64,reg:squarederror,rmse,0.199306,11,lossguide,475,305,0.572101,0.964780,0.631021,0.6,0.2
1,0.792407,64,reg:squarederror,rmse,0.017624,16,lossguide,427,230,0.595856,0.941849,0.675908,0.6,4.2
2,0.853171,64,reg:squarederror,rmse,0.683561,7,lossguide,488,386,0.757127,0.400145,0.731999,0.7,4.2
3,0.764726,64,reg:squarederror,rmse,0.116659,13,lossguide,280,321,0.852516,0.581348,0.936509,0.8,4.3
4,0.820941,64,reg:squarederror,rmse,0.630738,10,lossguide,171,324,0.762107,0.657339,0.964968,1.0,0.4


In [27]:
df.to_csv("data/immunopeptide_chymotrypsin_resultsB.csv")

In [28]:
params = {
    "nthread": 24,
    "objective": "reg:squarederror",
    #"objective": "binary:logistic",
    #"eval_metric": 'mae',
    "eval_metric": 'rmse',
    "verbosity": 1,
    "eta": 0.5,
    "max_depth": 9,
    "grow_policy":"lossguide",
    "max_leaves":100,
    "min_child_weight": 300,
    "gamma": 0.1,
    "subsample": 1,
    "lambda" : 0,
    "colsample_bytree": 1,
    "max_delta_step": 0,
}

In [29]:
params.update(best_hyperparams)

In [30]:
params["max_depth"] = int(params["max_depth"])
params["max_leaves"] = int(params["max_leaves"])
params["min_child_weight"] = int(params["min_child_weight"])
params

{'nthread': 24,
 'objective': 'reg:squarederror',
 'eval_metric': 'rmse',
 'verbosity': 1,
 'eta': 0.08060612330262913,
 'max_depth': 18,
 'grow_policy': 'lossguide',
 'max_leaves': 117,
 'min_child_weight': 500,
 'gamma': 0.031142279181653326,
 'subsample': 0.7000000000000001,
 'lambda': 0,
 'colsample_bytree': 0.891381182690278,
 'max_delta_step': 0,
 'reg_alpha': 3.4000000000000004,
 'reg_lambda': 0.27245538266226343}

In [31]:
bst = xgb.train(params, xtrain, 400, evallist, maximize=False, early_stopping_rounds=10)

[0]	train-rmse:8.69017	test-rmse:8.68801
[1]	train-rmse:7.99649	test-rmse:7.99719
[2]	train-rmse:7.36195	test-rmse:7.36278
[3]	train-rmse:6.78067	test-rmse:6.78068
[4]	train-rmse:6.24647	test-rmse:6.24583
[5]	train-rmse:5.75466	test-rmse:5.75498
[6]	train-rmse:5.30516	test-rmse:5.30451
[7]	train-rmse:4.89038	test-rmse:4.89110
[8]	train-rmse:4.51221	test-rmse:4.51234
[9]	train-rmse:4.16482	test-rmse:4.16514
[10]	train-rmse:3.84615	test-rmse:3.84695
[11]	train-rmse:3.55449	test-rmse:3.55548
[12]	train-rmse:3.28767	test-rmse:3.28884
[13]	train-rmse:3.04355	test-rmse:3.04501
[14]	train-rmse:2.82079	test-rmse:2.82216
[15]	train-rmse:2.61730	test-rmse:2.61883
[16]	train-rmse:2.43133	test-rmse:2.43352
[17]	train-rmse:2.26237	test-rmse:2.26468
[18]	train-rmse:2.10852	test-rmse:2.11132
[19]	train-rmse:1.96884	test-rmse:1.97188
[20]	train-rmse:1.84169	test-rmse:1.84516
[21]	train-rmse:1.72719	test-rmse:1.73104
[22]	train-rmse:1.62340	test-rmse:1.62768
[23]	train-rmse:1.53018	test-rmse:1.53502
[2

[194]	train-rmse:0.72337	test-rmse:0.77058
[195]	train-rmse:0.72306	test-rmse:0.77045
[196]	train-rmse:0.72290	test-rmse:0.77036
[197]	train-rmse:0.72269	test-rmse:0.77026
[198]	train-rmse:0.72252	test-rmse:0.77020
[199]	train-rmse:0.72220	test-rmse:0.77006
[200]	train-rmse:0.72210	test-rmse:0.77001
[201]	train-rmse:0.72183	test-rmse:0.76984
[202]	train-rmse:0.72147	test-rmse:0.76965
[203]	train-rmse:0.72112	test-rmse:0.76950
[204]	train-rmse:0.72084	test-rmse:0.76940
[205]	train-rmse:0.72053	test-rmse:0.76926
[206]	train-rmse:0.72032	test-rmse:0.76915
[207]	train-rmse:0.72007	test-rmse:0.76903
[208]	train-rmse:0.71974	test-rmse:0.76890
[209]	train-rmse:0.71937	test-rmse:0.76876
[210]	train-rmse:0.71912	test-rmse:0.76868
[211]	train-rmse:0.71886	test-rmse:0.76860
[212]	train-rmse:0.71860	test-rmse:0.76850
[213]	train-rmse:0.71838	test-rmse:0.76841
[214]	train-rmse:0.71828	test-rmse:0.76837
[215]	train-rmse:0.71800	test-rmse:0.76827
[216]	train-rmse:0.71765	test-rmse:0.76812
[217]	train

[385]	train-rmse:0.68457	test-rmse:0.75731
[386]	train-rmse:0.68440	test-rmse:0.75726
[387]	train-rmse:0.68418	test-rmse:0.75721
[388]	train-rmse:0.68402	test-rmse:0.75716
[389]	train-rmse:0.68387	test-rmse:0.75714
[390]	train-rmse:0.68366	test-rmse:0.75708
[391]	train-rmse:0.68350	test-rmse:0.75703
[392]	train-rmse:0.68341	test-rmse:0.75701
[393]	train-rmse:0.68327	test-rmse:0.75695
[394]	train-rmse:0.68320	test-rmse:0.75692
[395]	train-rmse:0.68310	test-rmse:0.75690
[396]	train-rmse:0.68304	test-rmse:0.75688
[397]	train-rmse:0.68278	test-rmse:0.75685
[398]	train-rmse:0.68265	test-rmse:0.75682
[399]	train-rmse:0.68250	test-rmse:0.75675


In [32]:
bst.save_model("Models/Immunopeptide_chymotrypsin_HCDb.xgboost")

# Fast b-ion model

In [34]:
space= {
    'eta': hp.loguniform('eta', np.log(0.01), np.log(1)),
    'max_depth': hp.quniform('max_depth', 2, 7, 1),
    'max_leaves': hp.quniform('max_leaves', 5, 500, 1),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1.0),
    'reg_lambda': hp.uniform('reg_lambda', 0.0, 1.0),
    'gamma' : hp.uniform ('gamma', 0.0,1),
    'min_child_weight' : hp.quniform('min_child_weight', 0, 500, 1),
    'subsample': hp.quniform('subsample', 0.5, 1 ,0.1),
    'reg_alpha': hp.quniform('reg_alpha', 0, 5 ,0.1),
    'grow_policy': hp.choice('grow_policy', ("lossguide", "depthwise"))
}

In [35]:
def objective(space):
    params = {
    "nthread": 64,
    "objective": "reg:squarederror",
    "eval_metric": 'rmse',
    "eta": space["eta"],
    "max_depth": int(space['max_depth']),
    "grow_policy":space['grow_policy'],
    "max_leaves":int(space["max_leaves"]),
    "min_child_weight": int(space["min_child_weight"]),
    "gamma": space['gamma'],
    "reg_lambda" : space['reg_lambda'],
    "colsample_bytree": space['colsample_bytree'],
    "subsample" : space["subsample"],
    "reg_alpha" : space["reg_alpha"]
    }
    print(params)
    tmp = xgb.cv(params, xtrain, nfold=4, num_boost_round=400, early_stopping_rounds=10, verbose_eval=True)
    rmse = tmp['test-rmse-mean'].min()

    print ("rmse:", rmse)
    return {'loss': rmse, 'status': STATUS_OK, "params": params}

In [36]:
trials = Trials()
best_hyperparams = fmin(fn = objective,
                        space = space,
                        algo = tpe.suggest,
                        max_evals = 30,
                        trials = trials)


best_hyperparams

{'nthread': 64, 'objective': 'reg:squarederror', 'eval_metric': 'rmse', 'eta': 0.016602908138118818, 'max_depth': 7, 'grow_policy': 'depthwise', 'max_leaves': 405, 'min_child_weight': 164, 'gamma': 0.050256953380391156, 'reg_lambda': 0.6812023260652089, 'colsample_bytree': 0.7988295968988938, 'subsample': 0.6000000000000001, 'reg_alpha': 3.9000000000000004}
[0]	train-rmse:9.28739+0.00035	test-rmse:9.28683+0.00106

[1]	train-rmse:9.13587+0.00033	test-rmse:9.13587+0.00106

[2]	train-rmse:8.98634+0.00034	test-rmse:8.98692+0.00104

[3]	train-rmse:8.84084+0.00032	test-rmse:8.84059+0.00102

[4]	train-rmse:8.69662+0.00025	test-rmse:8.69656+0.00106

[5]	train-rmse:8.55494+0.00023	test-rmse:8.55528+0.00105

[6]	train-rmse:8.41622+0.00027	test-rmse:8.41614+0.00105

[7]	train-rmse:8.27959+0.00024	test-rmse:8.27958+0.00103

[8]	train-rmse:8.14482+0.00023	test-rmse:8.14501+0.00100

[9]	train-rmse:8.01316+0.00025	test-rmse:8.01289+0.00098

[10]	train-rmse:7.88292+0.00029	test-rmse:7.88304+0.00089

[

[268]	train-rmse:1.00403+0.00016	test-rmse:1.00492+0.00105

[269]	train-rmse:1.00346+0.00014	test-rmse:1.00436+0.00105

[270]	train-rmse:1.00286+0.00013	test-rmse:1.00377+0.00103

[271]	train-rmse:1.00230+0.00018	test-rmse:1.00320+0.00108

[272]	train-rmse:1.00172+0.00017	test-rmse:1.00263+0.00106

[273]	train-rmse:1.00124+0.00020	test-rmse:1.00215+0.00109

[274]	train-rmse:1.00067+0.00017	test-rmse:1.00158+0.00112

[275]	train-rmse:1.00013+0.00011	test-rmse:1.00105+0.00116

[276]	train-rmse:0.99961+0.00019	test-rmse:1.00053+0.00120

[277]	train-rmse:0.99910+0.00013	test-rmse:1.00003+0.00112

[278]	train-rmse:0.99857+0.00016	test-rmse:0.99950+0.00116

[279]	train-rmse:0.99808+0.00011	test-rmse:0.99901+0.00111

[280]	train-rmse:0.99761+0.00013	test-rmse:0.99854+0.00116

[281]	train-rmse:0.99707+0.00009	test-rmse:0.99801+0.00112

[282]	train-rmse:0.99648+0.00011	test-rmse:0.99742+0.00112

[283]	train-rmse:0.99600+0.00013	test-rmse:0.99695+0.00110

[284]	train-rmse:0.99556+0.00017	test-rm

[0]	train-rmse:7.52804+0.00031	test-rmse:7.52777+0.00091                       

[1]	train-rmse:6.02016+0.00332	test-rmse:6.02031+0.00395                       

[2]	train-rmse:4.83104+0.00318	test-rmse:4.83117+0.00344                       

[3]	train-rmse:3.89881+0.00301	test-rmse:3.89886+0.00307                       

[4]	train-rmse:3.17375+0.00226	test-rmse:3.17378+0.00256                       

[5]	train-rmse:2.61359+0.00305	test-rmse:2.61361+0.00321                       

[6]	train-rmse:2.18362+0.00477	test-rmse:2.18372+0.00492                       

[7]	train-rmse:1.85994+0.00461	test-rmse:1.86006+0.00483                       

[8]	train-rmse:1.61991+0.00522	test-rmse:1.62011+0.00543                       

[9]	train-rmse:1.44396+0.00591	test-rmse:1.44420+0.00620                       

[10]	train-rmse:1.31853+0.00457	test-rmse:1.31881+0.00493                      

[11]	train-rmse:1.23144+0.00354	test-rmse:1.23176+0.00411                      

[12]	train-rmse:1.17019+0.00

[202]	train-rmse:0.85991+0.00071	test-rmse:0.86624+0.00140                     

[203]	train-rmse:0.85976+0.00074	test-rmse:0.86611+0.00141                     

[204]	train-rmse:0.85952+0.00073	test-rmse:0.86589+0.00137                     

[205]	train-rmse:0.85923+0.00068	test-rmse:0.86563+0.00127                     

[206]	train-rmse:0.85891+0.00069	test-rmse:0.86533+0.00123                     

[207]	train-rmse:0.85868+0.00062	test-rmse:0.86513+0.00120                     

[208]	train-rmse:0.85841+0.00066	test-rmse:0.86489+0.00124                     

[209]	train-rmse:0.85819+0.00067	test-rmse:0.86470+0.00122                     

[210]	train-rmse:0.85805+0.00069	test-rmse:0.86457+0.00126                     

[211]	train-rmse:0.85786+0.00067	test-rmse:0.86440+0.00128                     

[212]	train-rmse:0.85770+0.00071	test-rmse:0.86427+0.00134                     

[213]	train-rmse:0.85751+0.00076	test-rmse:0.86410+0.00137                     

[214]	train-rmse:0.85734+0.0

[0]	train-rmse:8.51484+0.00037	test-rmse:8.51499+0.00103                      

[1]	train-rmse:7.68352+0.00081	test-rmse:7.68371+0.00122                      

[2]	train-rmse:6.93725+0.00068	test-rmse:6.93737+0.00095                      

[3]	train-rmse:6.26806+0.00115	test-rmse:6.26814+0.00061                      

[4]	train-rmse:5.66846+0.00097	test-rmse:5.66842+0.00106                      

[5]	train-rmse:5.13170+0.00122	test-rmse:5.13171+0.00141                      

[6]	train-rmse:4.65145+0.00088	test-rmse:4.65141+0.00100                      

[7]	train-rmse:4.22211+0.00079	test-rmse:4.22216+0.00111                      

[8]	train-rmse:3.83914+0.00084	test-rmse:3.83917+0.00077                      

[9]	train-rmse:3.49770+0.00100	test-rmse:3.49774+0.00088                      

[10]	train-rmse:3.19434+0.00103	test-rmse:3.19439+0.00144                     

[11]	train-rmse:2.92467+0.00080	test-rmse:2.92470+0.00122                     

[12]	train-rmse:2.68641+0.00083	test-rms

...

[118]	train-rmse:0.87076+0.00169	test-rmse:0.87720+0.00232                      

[119]	train-rmse:0.87025+0.00180	test-rmse:0.87675+0.00241                      

[120]	train-rmse:0.86971+0.00178	test-rmse:0.87625+0.00238                      

[121]	train-rmse:0.86917+0.00160	test-rmse:0.87577+0.00218                      

[122]	train-rmse:0.86853+0.00153	test-rmse:0.87518+0.00222                      

[123]	train-rmse:0.86816+0.00150	test-rmse:0.87487+0.00218                      

[124]	train-rmse:0.86761+0.00156	test-rmse:0.87438+0.00213                      

[125]	train-rmse:0.86733+0.00159	test-rmse:0.87414+0.00213                      

[126]	train-rmse:0.86703+0.00155	test-rmse:0.87388+0.00208                      

[127]	train-rmse:0.86671+0.00156	test-rmse:0.87360+0.00205                      

[128]	train-rmse:0.86628+0.00149	test-rmse:0.87325+0.00194                      

[129]	train-rmse:0.86590+0.00157	test-rmse:0.87290+0.00196                      

[130]	train-rmse

[316]	train-rmse:0.82446+0.00072	test-rmse:0.83888+0.00103                      

[317]	train-rmse:0.82434+0.00068	test-rmse:0.83879+0.00101                      

[318]	train-rmse:0.82419+0.00072	test-rmse:0.83867+0.00097                      

[319]	train-rmse:0.82407+0.00071	test-rmse:0.83860+0.00095                      

[320]	train-rmse:0.82394+0.00071	test-rmse:0.83851+0.00097                      

[321]	train-rmse:0.82378+0.00070	test-rmse:0.83837+0.00097                      

[322]	train-rmse:0.82367+0.00071	test-rmse:0.83830+0.00098                      

[323]	train-rmse:0.82351+0.00069	test-rmse:0.83818+0.00099                      

[324]	train-rmse:0.82338+0.00065	test-rmse:0.83808+0.00099                      

[325]	train-rmse:0.82323+0.00067	test-rmse:0.83798+0.00103                      

[326]	train-rmse:0.82309+0.00069	test-rmse:0.83789+0.00107                      

[327]	train-rmse:0.82298+0.00072	test-rmse:0.83781+0.00109                      

[328]	train-rmse

[108]	train-rmse:1.18401+0.00111	test-rmse:1.18413+0.00074                      

[109]	train-rmse:1.18341+0.00116	test-rmse:1.18352+0.00075                      

[110]	train-rmse:1.18277+0.00119	test-rmse:1.18290+0.00077                      

[111]	train-rmse:1.18216+0.00119	test-rmse:1.18228+0.00086                      

[112]	train-rmse:1.18141+0.00113	test-rmse:1.18153+0.00091                      

[113]	train-rmse:1.18083+0.00109	test-rmse:1.18095+0.00091                      

[114]	train-rmse:1.18012+0.00107	test-rmse:1.18024+0.00085                      

[115]	train-rmse:1.17959+0.00103	test-rmse:1.17972+0.00090                      

[116]	train-rmse:1.17902+0.00100	test-rmse:1.17913+0.00088                      

[117]	train-rmse:1.17850+0.00109	test-rmse:1.17861+0.00098                      

[118]	train-rmse:1.17769+0.00093	test-rmse:1.17780+0.00089                      

[119]	train-rmse:1.17711+0.00091	test-rmse:1.17722+0.00081                      

[120]	train-rmse

[306]	train-rmse:1.11643+0.00149	test-rmse:1.11662+0.00065                      

[307]	train-rmse:1.11629+0.00140	test-rmse:1.11647+0.00059                      

[308]	train-rmse:1.11609+0.00139	test-rmse:1.11628+0.00059                      

[309]	train-rmse:1.11593+0.00140	test-rmse:1.11611+0.00059                      

[310]	train-rmse:1.11580+0.00143	test-rmse:1.11599+0.00061                      

[311]	train-rmse:1.11556+0.00148	test-rmse:1.11575+0.00063                      

[312]	train-rmse:1.11539+0.00148	test-rmse:1.11558+0.00060                      

[313]	train-rmse:1.11508+0.00138	test-rmse:1.11528+0.00050                      

[314]	train-rmse:1.11487+0.00140	test-rmse:1.11506+0.00053                      

[315]	train-rmse:1.11464+0.00139	test-rmse:1.11484+0.00052                      

[316]	train-rmse:1.11443+0.00129	test-rmse:1.11463+0.00046                      

[317]	train-rmse:1.11422+0.00131	test-rmse:1.11442+0.00050                      

[318]	train-rmse

{'colsample_bytree': 0.7848151878076199,
 'eta': 0.3781092476928985,
 'gamma': 0.06996074946187703,
 'grow_policy': 0,
 'max_depth': 7.0,
 'max_leaves': 178.0,
 'min_child_weight': 455.0,
 'reg_alpha': 4.3,
 'reg_lambda': 0.8000693484694821,
 'subsample': 0.8}

In [37]:
hyperoptimization_results = {'test-rmse-mean': [x['loss'] for x in trials.results]}
for key in trials.results[0]["params"].keys():
    hyperoptimization_results[key] = [x['params'][key] for x in trials.results]

In [38]:
df = pd.DataFrame(hyperoptimization_results)

In [39]:
df

,test-rmse-mean,nthread,objective,eval_metric,eta,max_depth,grow_policy,max_leaves,min_child_weight,gamma,reg_lambda,colsample_bytree,subsample,reg_alpha
0,0.959987,64,reg:squarederror,rmse,0.016603,7,depthwise,405,164,0.050257,0.681202,0.798830,0.6,3.9
1,0.839292,64,reg:squarederror,rmse,0.206926,7,depthwise,464,126,0.311932,0.187296,0.541700,0.7,0.6
2,0.932049,64,reg:squarederror,rmse,0.100318,5,lossguide,37,116,0.007688,0.243689,0.605985,0.5,1.9
3,0.859460,64,reg:squarederror,rmse,0.252189,6,depthwise,71,122,0.625440,0.446438,0.659243,0.6,4.4
4,0.894177,64,reg:squarederror,rmse,0.904584,5,lossguide,32,447,0.546149,0.154155,0.896689,0.6,1.5
5,0.960251,64,reg:squarederror,rmse,0.156704,4,depthwise,134,321,0.452511,0.700861,0.659554,0.5,2.0
6,1.146492,64,reg:squarederror,rmse,0.028222,3,lossguide,479,351,0.719716,0.234996,0.963862,0.7,1.6
7,0.962821,64,reg:squarederror,rmse,0.899945,3,lossguide,125,31,0.498392,0.955533,0.667469,0.6,2.0
8,1.010130,64,reg:squarederror,rmse,0.068371,4,lossguide,65,4,0.689045,0.152349,0.600160,1.0,3.2
9,0.945339,64,reg:squarederror,rmse,0.033636,6,depthwise,117,387,0.550947,0.629054,0.917246,0.7,0.8


In [40]:
df.to_csv("data/immunopeptide_chymotrypsin_resultsB.csv", mode="a", header=False)


In [41]:
params = {
    "nthread": 64,
    "objective": "reg:squarederror",
    #"objective": "binary:logistic",
    #"eval_metric": 'mae',
    "eval_metric": 'rmse',
    "verbosity": 1,
    "eta": 0.5,
    "max_depth": 9,
    "grow_policy":"lossguide",
    "max_leaves":100,
    "min_child_weight": 300,
    "gamma": 0.1,
    "subsample": 1,
    "reg_lambda" : 0,
    "colsample_bytree": 1,
}

In [42]:
best_hyperparams = space_eval(space, best_hyperparams)
params.update(best_hyperparams)

In [43]:
params["max_depth"] = int(params["max_depth"])
params["max_leaves"] = int(params["max_leaves"])
params["min_child_weight"] = int(params["min_child_weight"])


In [44]:
params

{'nthread': 64,
 'objective': 'reg:squarederror',
 'eval_metric': 'rmse',
 'verbosity': 1,
 'eta': 0.3781092476928985,
 'max_depth': 7,
 'grow_policy': 'lossguide',
 'max_leaves': 178,
 'min_child_weight': 455,
 'gamma': 0.06996074946187703,
 'subsample': 0.8,
 'reg_lambda': 0.8000693484694821,
 'colsample_bytree': 0.7848151878076199,
 'reg_alpha': 4.3}

In [45]:
bst2 = xgb.train(params, xtrain, 400, evallist, maximize=False, early_stopping_rounds=10)

[0]	train-rmse:5.95186	test-rmse:5.95062
[1]	train-rmse:3.81930	test-rmse:3.81917
[2]	train-rmse:2.54050	test-rmse:2.54090
[3]	train-rmse:1.81110	test-rmse:1.81196
[4]	train-rmse:1.41726	test-rmse:1.41861
[5]	train-rmse:1.21577	test-rmse:1.21763
[6]	train-rmse:1.12195	test-rmse:1.12414
[7]	train-rmse:1.07496	test-rmse:1.07737
[8]	train-rmse:1.05086	test-rmse:1.05331
[9]	train-rmse:1.03541	test-rmse:1.03796
[10]	train-rmse:1.02496	test-rmse:1.02744
[11]	train-rmse:1.01630	test-rmse:1.01868
[12]	train-rmse:1.01004	test-rmse:1.01235
[13]	train-rmse:1.00173	test-rmse:1.00425
[14]	train-rmse:0.99733	test-rmse:1.00000
[15]	train-rmse:0.99219	test-rmse:0.99485
[16]	train-rmse:0.98369	test-rmse:0.98645
[17]	train-rmse:0.97813	test-rmse:0.98082
[18]	train-rmse:0.97519	test-rmse:0.97782
[19]	train-rmse:0.97103	test-rmse:0.97369
[20]	train-rmse:0.96673	test-rmse:0.96948
[21]	train-rmse:0.96225	test-rmse:0.96508
[22]	train-rmse:0.95823	test-rmse:0.96103
[23]	train-rmse:0.95652	test-rmse:0.95934
[2

[194]	train-rmse:0.84357	test-rmse:0.85240
[195]	train-rmse:0.84350	test-rmse:0.85235
[196]	train-rmse:0.84342	test-rmse:0.85231
[197]	train-rmse:0.84328	test-rmse:0.85218
[198]	train-rmse:0.84289	test-rmse:0.85180
[199]	train-rmse:0.84274	test-rmse:0.85167
[200]	train-rmse:0.84241	test-rmse:0.85137
[201]	train-rmse:0.84229	test-rmse:0.85129
[202]	train-rmse:0.84215	test-rmse:0.85119
[203]	train-rmse:0.84186	test-rmse:0.85094
[204]	train-rmse:0.84169	test-rmse:0.85082
[205]	train-rmse:0.84152	test-rmse:0.85066
[206]	train-rmse:0.84111	test-rmse:0.85027
[207]	train-rmse:0.84090	test-rmse:0.85009
[208]	train-rmse:0.84082	test-rmse:0.85002
[209]	train-rmse:0.84067	test-rmse:0.84990
[210]	train-rmse:0.84050	test-rmse:0.84981
[211]	train-rmse:0.84029	test-rmse:0.84963
[212]	train-rmse:0.83998	test-rmse:0.84936
[213]	train-rmse:0.83979	test-rmse:0.84919
[214]	train-rmse:0.83960	test-rmse:0.84899
[215]	train-rmse:0.83943	test-rmse:0.84884
[216]	train-rmse:0.83923	test-rmse:0.84869
[217]	train

[385]	train-rmse:0.81535	test-rmse:0.83061
[386]	train-rmse:0.81525	test-rmse:0.83052
[387]	train-rmse:0.81513	test-rmse:0.83044
[388]	train-rmse:0.81502	test-rmse:0.83039
[389]	train-rmse:0.81498	test-rmse:0.83038
[390]	train-rmse:0.81486	test-rmse:0.83028
[391]	train-rmse:0.81477	test-rmse:0.83022
[392]	train-rmse:0.81472	test-rmse:0.83018
[393]	train-rmse:0.81451	test-rmse:0.83003
[394]	train-rmse:0.81426	test-rmse:0.82981
[395]	train-rmse:0.81408	test-rmse:0.82966
[396]	train-rmse:0.81403	test-rmse:0.82964
[397]	train-rmse:0.81390	test-rmse:0.82954
[398]	train-rmse:0.81374	test-rmse:0.82943
[399]	train-rmse:0.81362	test-rmse:0.82936


In [46]:
bst2.save_model("Models/Fast_immunopeptide_chymotrypsin_HCDb.xgboost")